In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPooling2D,
    Flatten,
    BatchNormalization,
    Dropout,
)

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
def plot_accuracy(history):
  plt.plot(history.history["accuracy"], label="train")
  plt.plot(history.history["val_accuracy"], label="validation")
  plt.legend()
  plt.show()

def plot_loss(history):
  plt.plot(history.history["loss"], label="train")
  plt.plot(history.history["val_loss"], label="validation")
  plt.legend()
  plt.show()

## The dataset can be downloaded from [Kaggle](https://www.kaggle.com/datasets/gpiosenka/sports-classification).

In [ ]:
train_ds = image_dataset_from_directory(
    directory="/content/drive/MyDrive/sportsdata/train",   
    label_mode="int",
    batch_size=64,
    image_size=(256, 256),
)

validation_ds = image_dataset_from_directory(
    directory="/content/drive/MyDrive/sportsdata/valid",   
    labels="inferred",
    label_mode="int",
    batch_size=64,
    image_size=(256, 256),
)

test_ds = image_dataset_from_directory(
    directory="/content/drive/MyDrive/sportsdata/test",     
    labels="inferred",
    label_mode="int",
    batch_size=64,
    image_size=(256, 256),
)

normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x,y: (normalization_layer(x),y))
validation_ds = validation_ds.map(lambda x,y: (normalization_layer(x),y))
test_ds = test_ds.map(lambda x,y: (normalization_layer(x),y))

###CUSTOM CNN

In [ ]:
model = Sequential()
model.add(Conv2D(128,kernel_size=(3, 3),padding="valid",activation="leaky_relu",input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
model.add(Conv2D(64, kernel_size=(3, 3), padding="valid", activation="leaky_relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
model.add(Conv2D(32, kernel_size=(3, 3), padding="valid", activation="leaky_relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
model.add(Flatten())
model.add(Dense(512, activation="leaky_relu"))
model.add(Dropout(0.1))
model.add(Dense(256, activation="leaky_relu"))
model.add(Dropout(0.1))
model.add(Dense(128, activation="leaky_relu"))
model.add(Dropout(0.1))
model.add(Dense(100, activation="softmax"))


model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

callbacks = [
tf.keras.callbacks.ModelCheckpoint("custom/sportsModelCustom_epoch{epoch}.h5"),
tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=2,
        verbose=1,
        restore_best_weights='True'
 ),
tf.keras.callbacks.TensorBoard('tb_logs/custom')
]

history = model.fit(
    train_ds,
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    validation_data=validation_ds,
)

plot_loss(history)
plot_accuracy(history)

### INCEPTION V3

In [ ]:

from tensorflow.keras.applications import InceptionV3

inception = InceptionV3(
    input_shape=(256, 256, 3), include_top=False, weights="imagenet", classes=100
)
inception.trainable = False
modelIncep = Sequential()
modelIncep.add(inception)
modelIncep.add(Flatten())
modelIncep.add(Dropout(0.1))
modelIncep.add(Dense(256, activation="leaky_relu"))
modelIncep.add(Dropout(0.1))
modelIncep.add(Dense(128, activation="leaky_relu"))
modelIncep.add(Dropout(0.1))
modelIncep.add(Dense(100, activation="softmax"))

modelIncep.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

callbacks = [
tf.keras.callbacks.ModelCheckpoint("incep/sportsModelInception_epoch{epoch}.h5"),
tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=2,
        verbose=1,
        restore_best_weights='True'
 ),
tf.keras.callbacks.TensorBoard('tb_logs/incep')
]

history = modelIncep.fit(
    train_ds,
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    validation_data=validation_ds,
)

plot_loss(history)
plot_accuracy(history)

## RESNET

In [ ]:
from tensorflow.keras.applications import ResNet50V2

resnet = ResNet50V2(
    input_shape=(256, 256, 3), include_top=False, weights="imagenet", classes=100
)
resnet.trainable = False
modelRes = Sequential()
modelRes.add(resnet)
modelRes.add(Flatten())
modelRes.add(Dropout(0.1))
modelRes.add(Dense(256, activation="leaky_relu"))
modelRes.add(Dropout(0.1))
modelRes.add(Dense(128, activation="leaky_relu"))
modelRes.add(Dropout(0.1))
modelRes.add(Dense(100, activation="softmax"))

modelRes.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

callbacks = [
tf.keras.callbacks.ModelCheckpoint("res/sportsModelResidual_epoch{epoch}.h5"),
tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=2,
        verbose=1,
        restore_best_weights='True'
 ),
tf.keras.callbacks.TensorBoard('tb_logs/res')
]

history = modelRes.fit(
    train_ds,
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    validation_data=validation_ds,
)

plot_loss(history)
plot_accuracy(history)

## MOBILENET

In [ ]:
from tensorflow.keras.applications import MobileNetV2

mobilenet = MobileNetV2(
    input_shape=(256, 256, 3), include_top=False, weights="imagenet", classes=100
)
mobilenet.trainable = True
modelMob = Sequential()
modelMob.add(mobilenet)
modelMob.add(Flatten())
modelMob.add(Dropout(0.1))
modelMob.add(Dense(256, activation="leaky_relu"))
modelMob.add(Dropout(0.1))
modelMob.add(Dense(128, activation="leaky_relu"))
modelMob.add(Dropout(0.1))
modelMob.add(Dense(100, activation="softmax"))

modelMob.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

callbacks = [
tf.keras.callbacks.ModelCheckpoint("mobile/sportsModelMobile_epoch{epoch}.h5"),
tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=2,
        verbose=1,
        restore_best_weights='True'
 ),
tf.keras.callbacks.TensorBoard('tb_logs/mobile')
]
history = modelMob.fit(
    train_ds,
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    validation_data=validation_ds,
)

plot_loss(history)
plot_accuracy(history)

## EFFICIENTNET

In [ ]:
from tensorflow.keras.applications import EfficientNetB3

efficientnet = EfficientNetB3(
    input_shape=(256, 256, 3), include_top=False, weights="imagenet", classes=100
)
efficientnet.trainable = True
modelEff = Sequential()
modelEff.add(efficientnet)
modelEff.add(Flatten())
modelEff.add(Dropout(0.1))
modelEff.add(Dense(256, activation="leaky_relu"))
modelEff.add(Dropout(0.1))
modelEff.add(Dense(128, activation="leaky_relu"))
modelEff.add(Dropout(0.1))
modelEff.add(Dense(100, activation="softmax"))

modelEff.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

callbacks = [
tf.keras.callbacks.ModelCheckpoint("efficient/sportsModelEfficient_epoch{epoch}.h5"),
tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=2,
        verbose=1,
        restore_best_weights='True'
 ),
tf.keras.callbacks.TensorBoard('tb_logs/efficient')
]
history = modelEff.fit(
    train_ds,
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    validation_data=validation_ds,
)

plot_loss(history)
plot_accuracy(history)

# PREDICTIONS

In [ ]:
model = tf.keras.models.load_model("sportsModelEfficient_epoch14.h5")

y_pred = np.array([])
y_true = np.array([])
for x, y in test_ds:
    y_pred = np.concatenate([y_pred, model.predict_classes(x)])
    y_true = np.concatenate([y_true, np.argmax(y.numpy(), axis=-1)])

print("Classification Report: \n", classification_report(y_pred, y_true))